In [7]:
import os
model_list = [  
                ['mistralai','Mistral-7B-v0.1'],
                ['meta-math','MetaMath-Mistral-7B'],
                ['itpossible','Chinese-Mistral-7B-v0.1'],
                ["HuggingFaceH4","zephyr-7b-beta"],
                ["cognitivecomputations","dolphin-2.6-mistral-7b"],
                ["meta-llama","Meta-Llama-3-8B"],
                ["cognitivecomputations","dolphin-2.9-llama3-8b"],
                ["meta-llama","Llama-2-13b-hf"],
                ]

output_file_path = "/data/home/chensh/projects/LLM_router/logs/voting_log/split_model7/voting_cache"
os.makedirs(output_file_path, exist_ok=True)

In [42]:
import json
import os
import time
import numpy as np
import random
random.seed(42)

# for mmlu (multiple choice problem)

task_list = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A. ', 'B. ', 'C. ', 'D. ']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/mmlu_5shot/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_mmlu_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            prediction = np.argmax(probability)
            answer = json_file[i]['doc']['answer']

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']["question"] + 
                                "".join(["\n" + choices_head[j] + choice for j, choice in enumerate(json_file[i]['doc']['choices'])]), "predictions": {model_pre+"/" + model: prediction}, "answer": answer}
                )
            else:
                output_data[count]["predictions"][model_pre+"/" + model] = prediction
                count += 1

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]

from collections import Counter

correct = 0
data_len = len(test_split)
for item_dict in test_split:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == int(item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)

2667
63.304058865416565


8951
63.74448084318474


In [43]:
import json
import os
import time
import numpy as np
import random
random.seed(52)

# modified_model_names = ["Meta-Llama-3-8B","Llama3-8B-Chinese-Chat","dolphin-2.9-llama3-8b"]
# for modified_model_name in modified_model_names:
#     path = os.path.join("/data/home/chensh/projects/LLM_router/output/cmmlu", modified_model_name)
#     files = os.listdir(path)
#     for file in files:
#         if file.startswith("pretrained"):
#             os.rename(os.path.join(path, file), os.path.join(path, file.replace(",max_model_len__4096", "")))    

# for cmmlu (multiple choice problem)

task_list = ['agronomy', 'anatomy', 'ancient_chinese', 'arts', 'astronomy', 'business_ethics', 'chinese_civil_service_exam', 'chinese_driving_rule', 'chinese_food_culture', 'chinese_foreign_policy', 'chinese_history', 'chinese_literature', 'chinese_teacher_qualification', 'clinical_knowledge', 'college_actuarial_science', 'college_education', 'college_engineering_hydrology', 'college_law', 'college_mathematics', 'college_medical_statistics', 'college_medicine', 'computer_science', 'computer_security', 'conceptual_physics', 'construction_project_management', 'economics', 'education', 'electrical_engineering', 'elementary_chinese', 'elementary_commonsense', 'elementary_information_and_technology', 'elementary_mathematics', 'ethnology', 'food_science', 'genetics', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_geography', 'high_school_mathematics', 'high_school_physics', 'high_school_politics', 'human_sexuality', 'international_law', 'journalism', 'jurisprudence', 'legal_and_moral_basis', 'logical', 'machine_learning', 'management', 'marketing', 'marxist_theory', 'modern_chinese', 'nutrition', 'philosophy', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_study', 'sociology', 'sports_science', 'traditional_chinese_medicine', 'virology', 'world_history', 'world_religions']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A', 'B', 'C', 'D']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/cmmlu/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_cmmlu_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            prediction = np.argmax(probability)
            answer = json_file[i]['target']

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']['Question'].strip() + "".join(["\n" + choices_head[j] + ". " + json_file[i]['doc'][choices_head[j]] for j in range(4)]) + "\n答案：", "predictions": {model_pre+"/" + model: prediction}, "answer": answer}
                )
            else:
                output_data[count]["predictions"][model_pre+"/" + model] = prediction
                count += 1

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]
                
from collections import Counter

correct = 0
data_len = len(test_split)
for item_dict in test_split:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == int(item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)


1650
47.482014388489205


In [44]:
import json
import os
import time
import numpy as np
import random
random.seed(100)

# for arc (multiple choice problem)

# task = "arc_easy"
task = "arc_challenge"

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/{task}/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
        json_file = json.load(f)
    for i in range(len(json_file)):
        resps = np.array(json_file[i]['resps'])
        log_probability = resps[:,0,0]
        probability = np.exp(log_probability)
        prediction = np.argmax(probability)
        answer = json_file[i]['target']

        if model_index == 0:
            output_data.append(
                {"question": "Question:" + json_file[i]['doc']["question"] + "\nAnswer:", "predictions": {model_pre+"/" + model: prediction}, "answer": answer}
            )
        else:
            output_data[count]["predictions"][model_pre+"/" + model] = prediction
            count += 1

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:int(0.7* len(output_data))]
test_split = output_data[int(0.7* len(output_data)):]

                
from collections import Counter

correct = 0
data_len = len(test_split)
for item_dict in test_split:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == int(item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)



179
50.85227272727273


In [45]:
import json
import os
import time
import numpy as np
import random       

# for ceval (multiple choice problem)
task_list = ['computer_network', 'operating_system', 'computer_architecture', 'college_programming', 'college_physics', 'college_chemistry', 'advanced_mathematics', 'probability_and_statistics', 'discrete_mathematics', 'electrical_engineer', 'metrology_engineer', 'high_school_mathematics', 'high_school_physics', 'high_school_chemistry', 'high_school_biology', 'middle_school_mathematics', 'middle_school_biology', 'middle_school_physics', 'middle_school_chemistry', 'veterinary_medicine', 'college_economics', 'business_administration', 'marxism', 'mao_zedong_thought', 'education_science', 'teacher_qualification', 'high_school_politics', 'high_school_geography', 'middle_school_politics', 'middle_school_geography', 'modern_chinese_history', 'ideological_and_moral_cultivation', 'logic', 'law', 'chinese_language_and_literature', 'art_studies', 'professional_tour_guide', 'legal_professional', 'high_school_chinese', 'high_school_history', 'middle_school_history', 'civil_servant', 'sports_science', 'plant_protection', 'basic_medicine', 'clinical_medicine', 'urban_and_rural_planner', 'accountant', 'fire_engineer', 'environmental_impact_assessment_engineer', 'tax_accountant', 'physician']

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    choices_head = ['A', 'B', 'C', 'D']
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/ceval-validation/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_ceval-valid_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = np.array(json_file[i]['resps'])
            log_probability = resps[:,0,0]
            probability = np.exp(log_probability)
            prediction = np.argmax(probability)
            answer = json_file[i]['target']

            if model_index == 0:
                output_data.append(
                    {"question":json_file[i]['doc']["question"] + 
                                "".join(["\n" + choices_head[j] + ". " + json_file[i]['doc'][choices_head[j]] for j in range(4)]) + "答案：", "predictions": {model_pre+"/" + model: prediction}, "answer": answer}
                )
            else:
                output_data[count]["predictions"][model_pre+"/" + model] = prediction
                count += 1

from collections import Counter

correct = 0
data_len = len(output_data)
for item_dict in output_data:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == int(item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)



654
48.588410104011885


In [48]:

import json
import os
import re
import time
import numpy as np

# for gsm8k

# task = "gsm8k_train"
task = "gsm8k-repeat10"

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/gsm8k_test-t0.2/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
        json_file = json.load(f)
    for i in range(len(json_file)):
        # extract answer_list & answer
        resps = json_file[i]['resps'][0]
        pattern = "#### (\\-?[0-9\\.\\,]+)"
        ans_list = [re.search(pattern, resp).group(1) if re.search(pattern, resp) else None for resp in resps]
        target = re.search(pattern, json_file[i]['target']).group(1)
        
        # compare with origin answer & get score
        if model_index == 0:
            output_data.append({"question":json_file[i]['doc']["question"], "predictions": {model_pre+"/" + model: ans_list[0]}, "answer": target})
        else:
            output_data[i]["predictions"][model_pre+"/" + model] = ans_list[0]
            
output_data = output_data[:int(len(output_data)/2)]

from collections import Counter

correct = 0
data_len = len(output_data)
for item_dict in output_data:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == (item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)


889
67.39954510993176


In [87]:

SUBSTITUTIONS = [
    ("an ", ""),
    ("a ", ""),
    (".$", "$"),
    ("\\$", ""),
    (r"\ ", ""),
    (" ", ""),
    ("mbox", "text"),
    (",\\text{and}", ","),
    ("\\text{and}", ","),
    ("\\text{m}", "\\text{}"),
]
REMOVED_EXPRESSIONS = [
    "square",
    "ways",
    "integers",
    "dollars",
    "mph",
    "inches",
    "ft",
    "hours",
    "km",
    "units",
    "\\ldots",
    "sue",
    "points",
    "feet",
    "minutes",
    "digits",
    "cents",
    "degrees",
    "cm",
    "gm",
    "pounds",
    "meters",
    "meals",
    "edges",
    "students",
    "childrentickets",
    "multiples",
    "\\text{s}",
    "\\text{.}",
    "\\text{\ns}",
    "\\text{}^2",
    "\\text{}^3",
    "\\text{\n}",
    "\\text{}",
    r"\mathrm{th}",
    r"^\circ",
    r"^{\circ}",
    r"\;",
    r",\!",
    "{,}",
    '"',
    "\\dots",
]


def get_unnormalized_answer(text: str) -> str:
    INVALID_ANSWER = "[invalidanswer]"
    end_seq = "I hope it is correct."
    text += end_seq
    match = re.search(
        r"Final Answer: The final answer is(.*?). I hope it is correct.",
        text,
    )
    if match:
        return match.group(1).strip()
    else:
        return INVALID_ANSWER
    

def normalize_final_answer(final_answer: str) -> str:
    """
    Normalize a final answer to a quantitative reasoning question.

    Copied character for character from appendix D of Lewkowycz et al. (2022)
    """
    final_answer = final_answer.split("=")[-1]

    for before, after in SUBSTITUTIONS:
        final_answer = final_answer.replace(before, after)
    for expr in REMOVED_EXPRESSIONS:
        final_answer = final_answer.replace(expr, "")

    # Extract answer that is in LaTeX math, is bold,
    # is surrounded by a box, etc.
    final_answer = re.sub(r"(.*?)(\$)(.*?)(\$)(.*)", "$\\3$", final_answer)
    final_answer = re.sub(r"(\\text\{)(.*?)(\})", "\\2", final_answer)
    final_answer = re.sub(r"(\\textbf\{)(.*?)(\})", "\\2", final_answer)
    final_answer = re.sub(r"(\\overline\{)(.*?)(\})", "\\2", final_answer)
    final_answer = re.sub(r"(\\boxed\{)(.*)(\})", "\\2", final_answer)

    # Normalize shorthand TeX:
    #  \fracab -> \frac{a}{b}
    #  \frac{abc}{bef} -> \frac{abc}{bef}
    #  \fracabc -> \frac{a}{b}c
    #  \sqrta -> \sqrt{a}
    #  \sqrtab -> sqrt{a}b
    final_answer = re.sub(r"(frac)([^{])(.)", "frac{\\2}{\\3}", final_answer)
    final_answer = re.sub(r"(sqrt)([^{])", "sqrt{\\2}", final_answer)
    final_answer = final_answer.replace("$", "")

    # Normalize 100,000 -> 100000
    if final_answer.replace(",", "").isdigit():
        final_answer = final_answer.replace(",", "")

    return final_answer



import json
import os
import time
import numpy as np
import random
random.seed(52)


# for cmmlu (multiple choice problem)

# task_list = ["minerva_math_precalc", "minerva_math_prealgebra", "minerva_math_num_theory", "minerva_math_intermediate_algebra", "minerva_math_geometry", "minerva_math_counting_and_prob", "minerva_math_algebra"]

task_list = ["minerva_math_prealgebra"]

output_data = []
for model_index, model_info in enumerate(model_list):
    count = 0
    for task in task_list:
        model_pre = model_info[0]
        model = model_info[1]
        with open(f"/data/home/chensh/projects/LLM_router/output/MATH/{model}/pretrained____data__home__chensh__data__huggingface_model__{model_pre}__{model}_{task}.jsonl", 'r') as f:
            json_file = json.load(f)
        for i in range(len(json_file)):
            resps = json_file[i]['resps'][0][0]
            unnormalized_answer = get_unnormalized_answer(resps)
            prediction = normalize_final_answer(unnormalized_answer)
            answer = json_file[i]['target']
            if model_index == 0:
                score = json_file[i]['exact_match']
                output_data.append(
                    {"question": json_file[i]['doc']['problem'], "predictions": {model_pre+"/" + model: prediction}, 'answer': answer}
                )
            else:
                output_data[count]["predictions"][model_pre+"/" + model] = prediction
                count += 1

from collections import Counter

correct = 0
data_len = len(output_data)
for item_dict in output_data:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    prediction_counter = Counter(predictions_list)
    final_predict, _ = prediction_counter.most_common()[0]
    if final_predict == (item_dict['answer']):
        correct += 1

print(correct)
print(correct / data_len * 100)

340
39.03559127439725


{'question': 'What is the smallest multiple of $5$ which is greater than $-32$?',
 'predictions': {'mistralai/Mistral-7B-v0.1': '-160',
  'meta-math/MetaMath-Mistral-7B': '-25',
  'itpossible/Chinese-Mistral-7B-v0.1': '35',
  'HuggingFaceH4/zephyr-7b-beta': '-30',
  'cognitivecomputations/dolphin-2.6-mistral-7b': '35',
  'meta-llama/Meta-Llama-3-8B': '5',
  'cognitivecomputations/dolphin-2.9-llama3-8b': '5',
  'meta-llama/Llama-2-13b-hf': '5'}}

In [79]:
from sacrebleu import sentence_bleu
import ast

def cut_def_question(func_code, question):
    len_question = len(question)
    return func_code[len_question:]


def most_similar_code(clist):
    cmp_res = lambda x, y: sentence_bleu(x, [y], lowercase=True).score
    if len(clist) == 1:
        return 0, clist[0], 0
    bleu_scores = []
    for idx, agent in enumerate(clist):
        total_score = 0
        for idx_o, otheragent in enumerate(clist):
            if idx == idx_o:
                continue
            total_score += cmp_res(agent, otheragent)
        bleu_scores.append(total_score)
    max_index, max_value = max(enumerate(bleu_scores), key=lambda x: x[1])
    return max_index, clist[max_index], max_value


import os
import json
import pandas as pd
import random
random.seed(42)

task = "humaneval"

humaneval_data = pd.read_parquet("/data/home/chensh/data/LLM_datasets/openai_humaneval/openai_humaneval/test-00000-of-00001.parquet",engine='pyarrow')

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/humaneval/{model}/log_samples.json",'r') as f:
        json_file = json.load(f)
    with open(f"/data/home/chensh/projects/LLM_router/output/humaneval/{model}/_humaneval.json",'r') as f:
        generation_file = json.load(f)
    json_file = json_file['humaneval']
    for index, instance in json_file.items():
        correct = 1 if instance[0][1]['passed'] else 0
        question = humaneval_data.iloc[int(index)]['prompt']
        prediction = generation_file[int(index)][0]
        prediction = cut_def_question(prediction, question)
        
        if model_index == 0:
            output_data.append({"question": question, "predictions": {model_pre + "/" + model: prediction}, "corrects": {model_pre + "/" + model: correct}} )
        else:
            output_data[int(index)]["predictions"][model_pre+"/" + model] = prediction
            output_data[int(index)]["corrects"][model_pre+"/" + model] = correct
            

train_split_index = random.sample(range(len(output_data)), len(output_data))
output_data = [output_data[index] for index in train_split_index]
train_split = output_data[:115]
test_split = output_data[115:]
# test_split = output_data[100:]


from collections import Counter
correct = 0
data_len = len(test_split)
for item_dict in test_split:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    max_index, _, _ =  most_similar_code(predictions_list)
    keys = list(item_dict['corrects'].keys())
    correct += item_dict['corrects'][keys[max_index]]

print(correct)
print(correct / data_len * 100)




21
42.857142857142854


In [91]:

# /data/home/chensh/projects/LLM_router/output/humaneval/Arithmo2-Mistral-7B_humaneval.json

import os
import json
import pandas as pd
import random
random.seed(42)

task = "mbpp"

mbpp_data = pd.read_parquet("/data/home/chensh/data/LLM_datasets/mbpp/full/test-00000-of-00001.parquet",engine='pyarrow')

output_data = []
for model_index, model_info in enumerate(model_list):
    model_pre = model_info[0]
    model = model_info[1]
    with open(f"/data/home/chensh/projects/LLM_router/output/mbpp/{model}/log_samples.json",'r') as f:
        json_file = json.load(f)
    with open(f"/data/home/chensh/projects/LLM_router/output/mbpp/{model}/_mbpp.json",'r') as f:
        generation_file = json.load(f)
    json_file = json_file['mbpp']
    for index, instance in json_file.items():
        correct = 1 if instance[0][1]['passed'] else 0
        description = mbpp_data.iloc[int(index)]["text"]
        test_example = mbpp_data.iloc[int(index)]["test_list"][0]
        question = f'"""\n{description}\n{test_example}\n"""\n'
        prediction = generation_file[int(index)][0]
        prediction = cut_def_question(prediction, question)
        
        if model_index == 0:
            output_data.append({"question": question, "predictions": {model_pre + "/" + model: prediction}, "corrects": {model_pre + "/" + model: correct}} )
        else:
            output_data[int(index)]["predictions"][model_pre+"/" + model] = prediction
            output_data[int(index)]["corrects"][model_pre+"/" + model] = correct
            

from collections import Counter
correct = 0
data_len = len(output_data)
for item_dict in output_data:
    predictions = item_dict['predictions']
    predictions_list = list(predictions.values())
    max_index, _, _ =  most_similar_code(predictions_list)
    keys = list(item_dict['corrects'].keys())
    correct += item_dict['corrects'][keys[max_index]]

print(correct)
print(correct / data_len * 100)



208
41.6


In [51]:
with open(f"/data/home/chensh/projects/LLM_router/output/humaneval/Meta-Llama-3-8B/_humaneval.json",'r') as f:
    generation_file = json.load(f)
generation_file[0][0]

'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n\n\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n'

In [74]:
# print(test_split[0]['predictions']['mistralai/Mistral-7B-v0.1'])
# answer = cut_def_question(test_split[0]['predictions']['mistralai/Mistral-7B-v0.1'], test_split[0]['question'])
# print(answer)

sentence_bleu(test_split[0]['predictions']['mistralai/Mistral-7B-v0.1'], [test_split[0]['predictions']['itpossible/Chinese-Mistral-7B-v0.1']], lowercase=True).score
# print(answer)

84.03034716144347

In [75]:
sentence_bleu(test_split[0]['predictions']['mistralai/Mistral-7B-v0.1'], [test_split[0]['predictions']['meta-llama/Meta-Llama-3-8B']], lowercase=True).score

55.880375741396136